In [61]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

In [88]:
binned = pd.read_csv('/Users/flatironschool/Absenteeism_Project/data/processed/combo_cleaned.csv')

In [83]:
binned.head()

,Unnamed: 0,Unnamed: 0.1,STNAM,LEANM,NCESSCH,SCHNAM,ALL_COHORT_1516,ALL_RATE_1516,LEA_STATE,LEA_STATE_NAME,...,sports_part,sports_rate,total_suspension_days,suspensed_day_rate,harassed,harassed_rate,activities_funds_rate,non_cert_rate,counselor_rate,absent_teacher_rate
0,0,0,ALABAMA,Albertville City,10000500871,Albertville High Sch,296,92,AL,ALABAMA,...,220.0,0.169884,78.0,0.060232,0.0,0.000000,2811.937359,0.0,0.003475,0.378788
1,1,1,ALABAMA,Marshall County,10000600872,Asbury Sch,67,GE95,AL,ALABAMA,...,175.0,0.324675,10.0,0.018553,2.0,0.003711,4825.189777,0.0,0.002783,0.100000
2,2,2,ALABAMA,Marshall County,10000600878,Douglas High Sch,153,85-89,AL,ALABAMA,...,229.0,0.385522,18.0,0.030303,5.0,0.008418,5317.932795,0.0,0.001684,0.105263
3,3,3,ALABAMA,Marshall County,10000600883,Kate D Smith DAR High Sch,120,80-84,AL,ALABAMA,...,241.0,0.525054,10.0,0.021786,0.0,0.000000,5909.375686,0.0,0.002179,0.068966
4,4,4,ALABAMA,Marshall County,10000601585,Brindlee Mt High Sch,94,85-89,AL,ALABAMA,...,64.0,0.176309,8.0,0.022039,0.0,0.000000,3962.305785,0.0,0.002755,0.101010


## Imputed mean for missing values 

In [84]:
#impute mean for numerical vars
#did not include total enrollment or all rate
binned_feat = binned[['ap_ib_de_rate', 'sat_act_rate', 'pass_algebra_rate',
      'geometry_rate', 'algebra2_rate', 'calc_rate', 'chronic_absent_rate', 'activities_funds_rate',
      'sports_rate', 'suspensed_day_rate', 'harassed_rate',
      'non_cert_rate','counselor_rate','absent_teacher_rate']]

In [85]:
for i in range(0, len(binned_feat.columns)):
    print('im here:', binned_feat.columns[i])
    binned_feat.iloc[:,i].replace(np.NaN, binned_feat.iloc[:,i].mean(), inplace=True)
    

im here: ap_ib_de_rate
im here: sat_act_rate
im here: pass_algebra_rate
im here: geometry_rate
im here: algebra2_rate
im here: calc_rate
im here: chronic_absent_rate
im here: activities_funds_rate
im here: sports_rate
im here: suspensed_day_rate
im here: harassed_rate
im here: non_cert_rate
im here: counselor_rate
im here: absent_teacher_rate


In [81]:
binned_feat.isna().sum()

ap_ib_de_rate            0
sat_act_rate             0
pass_algebra_rate        0
geometry_rate            0
algebra2_rate            0
calc_rate                0
chronic_absent_rate      0
activities_funds_rate    0
sports_rate              0
suspensed_day_rate       0
harassed_rate            0
non_cert_rate            0
counselor_rate           0
absent_teacher_rate      0
dtype: int64

## Binning Graduation Rates into 10 Rates



In [95]:
binned.head()

,Unnamed: 0,Unnamed: 0.1,STNAM,LEANM,NCESSCH,SCHNAM,ALL_COHORT_1516,ALL_RATE_1516,LEA_STATE,LEA_STATE_NAME,...,sports_part,sports_rate,total_suspension_days,suspensed_day_rate,harassed,harassed_rate,activities_funds_rate,non_cert_rate,counselor_rate,absent_teacher_rate
0,0,0,ALABAMA,Albertville City,10000500871,Albertville High Sch,296,92,AL,ALABAMA,...,220.0,0.169884,78.0,0.060232,0.0,0.000000,2811.937359,0.0,0.003475,0.378788
1,1,1,ALABAMA,Marshall County,10000600872,Asbury Sch,67,GE95,AL,ALABAMA,...,175.0,0.324675,10.0,0.018553,2.0,0.003711,4825.189777,0.0,0.002783,0.100000
2,2,2,ALABAMA,Marshall County,10000600878,Douglas High Sch,153,85-89,AL,ALABAMA,...,229.0,0.385522,18.0,0.030303,5.0,0.008418,5317.932795,0.0,0.001684,0.105263
3,3,3,ALABAMA,Marshall County,10000600883,Kate D Smith DAR High Sch,120,80-84,AL,ALABAMA,...,241.0,0.525054,10.0,0.021786,0.0,0.000000,5909.375686,0.0,0.002179,0.068966
4,4,4,ALABAMA,Marshall County,10000601585,Brindlee Mt High Sch,94,85-89,AL,ALABAMA,...,64.0,0.176309,8.0,0.022039,0.0,0.000000,3962.305785,0.0,0.002755,0.101010


In [96]:
#separate out schools that have 31 or more students in the graduation class
bigger = binned[binned['ALL_COHORT_1516'] >= 31]

In [97]:
#remove "GE" and "LE" from ranges
bigger.replace({'ALL_RATE_1516': 'GE95'}, '95', inplace=True)
bigger.replace({'ALL_RATE_1516': 'GE90'}, '90', inplace=True)
bigger.replace({'ALL_RATE_1516': 'GE99'}, '99', inplace=True)
bigger.replace({'ALL_RATE_1516': 'LE10'}, '10', inplace=True)
bigger.replace({'ALL_RATE_1516': 'LE1'}, '1', inplace=True)
bigger.replace({'ALL_RATE_1516': 'LE5'}, '5', inplace=True)


/anaconda3/envs/absenteeism/lib/python3.6/site-packages/pandas/core/frame.py:3798: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


In [98]:
#smallest range needs to be dealt with, has one digit before '-'
bigger.replace({'ALL_RATE_1516': '6-9'}, '6', inplace=True)

/anaconda3/envs/absenteeism/lib/python3.6/site-packages/pandas/core/frame.py:3798: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


In [99]:
#take first two digits of rates
bigger['grad_slice'] = bigger['ALL_RATE_1516'].str[:2]

/anaconda3/envs/absenteeism/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [100]:
bigger.reset_index(inplace=True)

In [101]:
bigger.head()

,index,Unnamed: 0,Unnamed: 0.1,STNAM,LEANM,NCESSCH,SCHNAM,ALL_COHORT_1516,ALL_RATE_1516,LEA_STATE,...,sports_rate,total_suspension_days,suspensed_day_rate,harassed,harassed_rate,activities_funds_rate,non_cert_rate,counselor_rate,absent_teacher_rate,grad_slice
0,0,0,0,ALABAMA,Albertville City,10000500871,Albertville High Sch,296,92,AL,...,0.169884,78.0,0.060232,0.0,0.000000,2811.937359,0.0,0.003475,0.378788,92
1,1,1,1,ALABAMA,Marshall County,10000600872,Asbury Sch,67,95,AL,...,0.324675,10.0,0.018553,2.0,0.003711,4825.189777,0.0,0.002783,0.100000,95
2,2,2,2,ALABAMA,Marshall County,10000600878,Douglas High Sch,153,85-89,AL,...,0.385522,18.0,0.030303,5.0,0.008418,5317.932795,0.0,0.001684,0.105263,85
3,3,3,3,ALABAMA,Marshall County,10000600883,Kate D Smith DAR High Sch,120,80-84,AL,...,0.525054,10.0,0.021786,0.0,0.000000,5909.375686,0.0,0.002179,0.068966,80
4,4,4,4,ALABAMA,Marshall County,10000601585,Brindlee Mt High Sch,94,85-89,AL,...,0.176309,8.0,0.022039,0.0,0.000000,3962.305785,0.0,0.002755,0.101010,85


In [102]:
bigger['grad_rate_bin'] = pd.cut(bigger['grad_slice'].astype(int), [0, 9, 19, 29, 39, 49, 59, 69, 79, 89, 100],
      labels = ['0-9%', '10-19%', '20-29%', '30-39%', '40-49%', '50-59%',
               '60-69%', '70-79%', '80-89%', '90%+'])

/anaconda3/envs/absenteeism/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [103]:
bigger.tail()

,index,Unnamed: 0,Unnamed: 0.1,STNAM,LEANM,NCESSCH,SCHNAM,ALL_COHORT_1516,ALL_RATE_1516,LEA_STATE,...,total_suspension_days,suspensed_day_rate,harassed,harassed_rate,activities_funds_rate,non_cert_rate,counselor_rate,absent_teacher_rate,grad_slice,grad_rate_bin
16568,21854,21854,21854,WYOMING,Sheridan County School District #1,560569000311,Big Horn High School,37,90,WY,...,2.0,0.013245,4.0,0.026490,1738.913907,0.0,0.006623,0.846154,90,90%+
16569,21858,21858,21858,WYOMING,Sheridan County School District #2,560569500360,Sheridan High School,236,89,WY,...,46.0,0.046796,0.0,0.000000,1694.559176,0.0,0.005086,0.198779,89,80-89%
16570,21861,21861,21861,WYOMING,Sweetwater County School District #2,560576200324,Green River High School,176,85-89,WY,...,110.0,0.141570,1.0,0.001287,1840.877490,0.0,0.003861,0.200000,85,80-89%
16571,21863,21863,21863,WYOMING,Teton County School District #1,560583000335,Jackson Hole High School,127,95,WY,...,26.0,0.040625,0.0,0.000000,4153.518984,0.0,0.004687,0.118939,95,90%+
16572,21866,21866,21866,WYOMING,Washakie County School District #1,560624000343,Worland High School,105,75-79,WY,...,10.0,0.025000,0.0,0.000000,3015.755325,0.0,0.004275,0.331544,75,70-79%


In [104]:
bigger['grad_rate_bin'].value_counts()

90%+      8712
80-89%    4257
70-79%    1457
60-69%     536
50-59%     352
10-19%     305
40-49%     278
30-39%     264
20-29%     264
0-9%       148
Name: grad_rate_bin, dtype: int64

In [105]:
#Juvenile Justice schools have very different graduation rates, 
#31 of 41 schools have graduation rates of less than 50%.

byRate = bigger[bigger['JJ'] == 'Yes'].groupby('grad_rate_bin')

In [106]:
byRate.count()

,index,Unnamed: 0,Unnamed: 0.1,STNAM,LEANM,NCESSCH,SCHNAM,ALL_COHORT_1516,ALL_RATE_1516,LEA_STATE,...,sports_rate,total_suspension_days,suspensed_day_rate,harassed,harassed_rate,activities_funds_rate,non_cert_rate,counselor_rate,absent_teacher_rate,grad_slice
grad_rate_bin,,,,,,,,,,,,,,,,,,,,,
0-9%,8,8,8,8,8,8,8,8,8,8,...,0,8,8,8,8,8,8,8,8,8
10-19%,15,15,15,15,15,15,15,15,15,15,...,0,15,15,15,15,14,15,15,15,15
20-29%,7,7,7,7,7,7,7,7,7,7,...,0,7,7,7,7,7,7,7,7,7
30-39%,5,5,5,5,5,5,5,5,5,5,...,0,5,5,5,5,5,5,5,5,5
40-49%,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50-59%,3,3,3,3,3,3,3,3,3,3,...,0,3,3,3,3,3,3,3,3,3
60-69%,1,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,1
70-79%,1,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,1
80-89%,1,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,1


In [107]:
#Alternative Schools have a different graduation rate pattern. Out of 
#1,055 Alternative Schools, 629 graduate less than half of their cohort.
byRate_alt = bigger[bigger['SCH_STATUS_ALT'] == 'Yes'].groupby('grad_rate_bin')
byRate_alt.count()

,index,Unnamed: 0,Unnamed: 0.1,STNAM,LEANM,NCESSCH,SCHNAM,ALL_COHORT_1516,ALL_RATE_1516,LEA_STATE,...,sports_rate,total_suspension_days,suspensed_day_rate,harassed,harassed_rate,activities_funds_rate,non_cert_rate,counselor_rate,absent_teacher_rate,grad_slice
grad_rate_bin,,,,,,,,,,,,,,,,,,,,,
0-9%,85,85,85,85,85,85,85,85,85,85,...,2,85,85,84,84,48,61,85,61,85
10-19%,162,162,162,162,162,162,162,162,162,162,...,7,162,162,160,160,125,146,162,146,162
20-29%,128,128,128,128,128,128,128,128,128,128,...,6,128,128,127,127,119,125,128,125,128
30-39%,128,128,128,128,128,128,128,128,128,128,...,10,128,128,127,127,123,126,128,126,128
40-49%,126,126,126,126,126,126,126,126,126,126,...,5,126,126,123,123,124,126,126,126,126
50-59%,106,106,106,106,106,106,106,106,106,106,...,7,106,106,106,106,106,105,106,105,106
60-69%,97,97,97,97,97,97,97,97,97,97,...,2,97,97,95,95,95,96,97,96,97
70-79%,96,96,96,96,96,96,96,96,96,96,...,3,96,96,95,95,93,94,96,94,96
80-89%,79,79,79,79,79,79,79,79,79,79,...,4,79,79,79,79,79,79,79,79,79


In [108]:
#The 75 Special Education Schools have a different graduation rate pattern. Out of 
#30 Special Education Schools graduate less than half of their cohort; however,
#25 Special Education Schools graduate 90% or more of their cohort.
byRate_sped = bigger[bigger['SCH_STATUS_SPED'] == 'Yes'].groupby('grad_rate_bin')
byRate_sped.count()

,index,Unnamed: 0,Unnamed: 0.1,STNAM,LEANM,NCESSCH,SCHNAM,ALL_COHORT_1516,ALL_RATE_1516,LEA_STATE,...,sports_rate,total_suspension_days,suspensed_day_rate,harassed,harassed_rate,activities_funds_rate,non_cert_rate,counselor_rate,absent_teacher_rate,grad_slice
grad_rate_bin,,,,,,,,,,,,,,,,,,,,,
0-9%,3,3,3,3,3,3,3,3,3,3,...,0,3,3,3,3,3,3,3,3,3
10-19%,16,16,16,16,16,16,16,16,16,16,...,1,16,16,16,16,15,15,16,15,16
20-29%,3,3,3,3,3,3,3,3,3,3,...,0,3,3,3,3,3,3,3,3,3
30-39%,3,3,3,3,3,3,3,3,3,3,...,0,3,3,3,3,2,3,3,3,3
40-49%,5,5,5,5,5,5,5,5,5,5,...,2,5,5,5,5,5,5,5,5,5
50-59%,2,2,2,2,2,2,2,2,2,2,...,0,2,2,2,2,2,2,2,2,2
60-69%,4,4,4,4,4,4,4,4,4,4,...,2,4,4,4,4,3,4,4,4,4
70-79%,5,5,5,5,5,5,5,5,5,5,...,4,5,5,5,5,5,5,5,5,5
80-89%,9,9,9,9,9,9,9,9,9,9,...,7,9,9,9,9,9,9,9,9,9


## Model 1 - Random Forest Classifier 

In [111]:
#impute mean for numerical vars
#did not include total enrollment or all rate
bigger_feat = bigger[['ap_ib_de_rate', 'sat_act_rate', 'pass_algebra_rate',
      'geometry_rate', 'algebra2_rate', 'calc_rate', 'chronic_absent_rate', 'activities_funds_rate',
      'sports_rate', 'suspensed_day_rate', 'harassed_rate',
      'non_cert_rate','counselor_rate','absent_teacher_rate']]

In [113]:
for i in range(0, len(bigger_feat.columns)):
    print('im here:', bigger_feat.columns[i])
    bigger.fillna(bigger.mean(), inplace=True)
    

im here: ap_ib_de_rate


/anaconda3/envs/absenteeism/lib/python3.6/site-packages/pandas/core/generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


im here: sat_act_rate
im here: pass_algebra_rate
im here: geometry_rate
im here: algebra2_rate
im here: calc_rate
im here: chronic_absent_rate
im here: activities_funds_rate
im here: sports_rate
im here: suspensed_day_rate
im here: harassed_rate
im here: non_cert_rate
im here: counselor_rate
im here: absent_teacher_rate


In [114]:
y = bigger['grad_rate_bin']
X = bigger[['ap_ib_de_rate', 'sat_act_rate', 'pass_algebra_rate',
      'geometry_rate', 'algebra2_rate', 'calc_rate', 'chronic_absent_rate', 'activities_funds_rate',
      'sports_rate', 'suspensed_day_rate', 'harassed_rate',
      'non_cert_rate','counselor_rate','absent_teacher_rate']]

In [115]:
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X, y, test_size=0.33, random_state=42)

In [116]:
rf = RandomForestClassifier(n_estimators=100, max_depth=5)

In [118]:
rf.fit(X_train_rf, y_train_rf)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [120]:
rf.score(X_test_rf, y_test_rf)

0.5544789762340037

In [121]:
rf.feature_importances_

array([0.0021053 , 0.07747109, 0.00467865, 0.01376386, 0.020208  ,
       0.10701053, 0.2858939 , 0.01075091, 0.09482216, 0.28656605,
       0.00424714, 0.01972153, 0.05307932, 0.01968156])